# PARTE III: Preparación de los datos climatológicos

In [1]:
from bs4 import BeautifulSoup 
import requests 

Exportamos los datos de la página del GHCN y los guardamos en la lista `lista_de_datos`

In [3]:
url_page='https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/access/'

In [4]:
page=requests.get(url_page).text
soup=BeautifulSoup(page)

In [5]:
links = [link.get('href') for link in soup.find_all('a') 
                          if 'MXN00014' in link.get('href')]

In [6]:
len(links)

251

In [7]:
import io 
import pandas as pd

La siguiente celda tarda en ejecutarse de 10 a 20 minutos.

In [8]:
lista_de_datos=[]
for link in links:
    url = url_page + link
    s= requests.get(url).content
    datos=pd.read_csv(io.StringIO(s.decode('utf-8')), header=0, index_col=1)
    lista_de_datos.append(datos)

Definimos la función `datos_2010(n)` que tiene como parámetro el número de la estación n y nos devuelve el número de registros del año 2010, el periodo que abarcó la captura de datos y el promedio de precipitación, temperatura máxima y mínima del año 2010. Evitamos trabajar con décimas (así vienen de la página del GHCN), por lo que dividimos entre 10 los promedios. 

In [9]:
def datos_2010(n):
    primer_dia=lista_de_datos[n].index[0]
    ultimo_dia=lista_de_datos[n].index[-1]
    datos_2010=0
    for date in lista_de_datos[n].loc[primer_dia:ultimo_dia].index:
        if '2010-12-31'>=date>='2010-01-01' :
            datos_2010+=1
    
    datos_meteorologicos=lista_de_datos[n].loc['2010-01-01':'2010-12-31',['PRCP','TMAX','TMIN']].mean()/10
    
    estacion_nombre=lista_de_datos[n].loc[primer_dia,['STATION','NAME']]
    return datos_2010, datos_meteorologicos, primer_dia, ultimo_dia, estacion_nombre


Explorando los datos de las estaciones, encontramos que se dividen en 3 tipos: 
1. las que cuentan con todos los registros del año 2010
2. las que no tienen ningún registro del año 2010
3. las que tienen registros de algunos días del año 2010

A continuación mostramos cómo registramos los datos para cualquier tipo de municipio

## 1. Estaciones que cuentan con registros de todos los días del año 2010

Como no tienen ningún dato faltante del año 2010, simplemente se asigna el promedio anual de precipitación temperatura máxima y temperatura mínima del año 2010. 
### Ejemplo
Este es el caso de Atoyac

In [10]:
datos_2010(16)

(365,
 PRCP     2.301370
 TMAX    30.070959
 TMIN    12.045753
 dtype: float64,
 '1943-08-01',
 '2012-01-31',
 STATION    MXN00014018
 NAME        ATOYAC, MX
 Name: 1943-08-01, dtype: object)

## 2.  Estaciones que no tienen ningún registro del año 2010

Le asignamos los valores climáticos de una estación cercana. Calculamos las geodésicas de dos puntos sobre la Tierra determinados por su latitud y longitud con la librería geopy que, por cierto, no soporta trabajar con altitudes diferentes. Escribimos la función `k_estaciones_cercanas(p1,k)` que encuentra k estaciones cercanas a dicho municipio. Sus parametros: longitud, latitud, altitud (que determinan el punto sobre la Tierra p1) y el número k.  

In [11]:
def k_estaciones_cercanas(p1,k):
    p1_lat_lon=geopy.point.Point(p1[[0,1]])
    distancias=[]
    for i in range(251):
        p2=geopy.point.Point(X[i,[0,1]])
        distancia=geopy.distance.geodesic(p1_lat_lon, p2).km
        distancias.append(distancia)
    distancias=np.array(distancias)
    k_estaciones_cercanas_ind=distancias.argsort()[:k]
    
    delta_alts=[]
    for i in list(k_estaciones_cercanas_ind):
        delta_alt=np.abs(p1[2]-X[i,2])
        delta_alts.append(delta_alt)
    delta_alts=np.array(delta_alts)
    delta_alts_ind=delta_alts.argsort()[:k]
    k_estaciones_cercanas_ind=list(k_estaciones_cercanas_ind)
    delta_alts_ind=list(delta_alts_ind)
    k_estaciones_cercanas_ordenadas=[]
    for i in delta_alts_ind:
        k_estaciones_cercanas_ordenadas.append(k_estaciones_cercanas_ind[i])
        
    return k_estaciones_cercanas_ordenadas

### Ejemplo
Acatic es un municipio cuya estación no tiene ningún registro del año 2010

In [12]:
datos_2010(0)

(0,
 PRCP   NaN
 TMAX   NaN
 TMIN   NaN
 dtype: float64,
 '1961-01-01',
 '1973-12-31',
 STATION       MXN00014001
 NAME       ACATIC SMN, MX
 Name: 1961-01-01, dtype: object)

## Estaciones meteorológicas

In [13]:
import geopy

In [14]:
from geopy import distance

In [15]:
import numpy as np

In [16]:
stations=pd.read_fwf('datos/ghcnd-stations.txt', header=None) # informacion de las estaciones: 

In [17]:
stations.columns=['station ID','latitude', 'longitude', 'elevation', 'State','a','b','c']

In [18]:
stations.loc[42682:42932]

,station ID,latitude,longitude,elevation,State,a,b,c
42682,MXN00014001,20.7833,102.9167,1677.9,ACATIC (SMN),NaN,NaN,NaN
42683,MXN00014002,20.4167,103.5833,1369.8,ACATLAN DE JUAREZ,NaN,NaN,NaN
42684,MXN00014004,21.3500,102.3333,1809.9,AGOSTADERO,NaN,NaN,NaN
42685,MXN00014005,19.0833,103.1000,819.9,AHUIJULLO,NaN,NaN,NaN
42686,MXN00014006,21.5500,102.4333,1745.0,AJOJUCAR,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
42928,MXN00014391,20.0500,103.1000,1889.8,PRESA EL VOLANTIN,NaN,NaN,NaN
42929,MXN00014392,21.4833,101.7500,1998.0,PASO DEL CUARENTA II,NaN,NaN,NaN
42930,MXN00014395,19.9833,104.1833,1460.0,IXTLAHUACAN DE SANTIAGO,NaN,NaN,NaN
42931,MXN00014396,20.2667,103.4167,1529.8,PIEDRA BARRENADA,NaN,NaN,NaN


In [19]:
stations.drop(['a','b','c'], axis=1, inplace=True)

In [20]:
stations.drop(stations.loc[:42681].index, axis=0, inplace=True)

In [21]:
stations.drop(stations.loc[42933:].index, axis=0, inplace=True)

In [22]:
stations.reset_index(drop=True, inplace=True)

In [23]:
stations

,station ID,latitude,longitude,elevation,State
0,MXN00014001,20.7833,102.9167,1677.9,ACATIC (SMN)
1,MXN00014002,20.4167,103.5833,1369.8,ACATLAN DE JUAREZ
2,MXN00014004,21.3500,102.3333,1809.9,AGOSTADERO
3,MXN00014005,19.0833,103.1000,819.9,AHUIJULLO
4,MXN00014006,21.5500,102.4333,1745.0,AJOJUCAR
...,...,...,...,...,...
246,MXN00014391,20.0500,103.1000,1889.8,PRESA EL VOLANTIN
247,MXN00014392,21.4833,101.7500,1998.0,PASO DEL CUARENTA II
248,MXN00014395,19.9833,104.1833,1460.0,IXTLAHUACAN DE SANTIAGO
249,MXN00014396,20.2667,103.4167,1529.8,PIEDRA BARRENADA


In [24]:
X=np.array(stations[['latitude','longitude','elevation']]) #solo necesitamos esta información de las estaciones

### Regresando al ejemplo de Acatic

In [25]:
k_estaciones_cercanas(X[0,:],5)

[0, 196, 96, 250, 81]

In [26]:
stations.loc[[0, 196, 96, 250, 81]]

,station ID,latitude,longitude,elevation,State
0,MXN00014001,20.7833,102.9167,1677.9,ACATIC (SMN)
196,MXN00014307,20.7667,102.9000,1679.8,ACATIC (DGE)
96,MXN00014104,20.6333,102.9500,1729.7,PALO VERDE
250,MXN00014397,20.6833,102.9500,1619.7,PRESA CALDERON
81,MXN00014087,20.7167,102.8000,1773.9,LA RED


La estación más cercana con datos es la de Palo Verde.   

Cuando ninguna de las estaciones cercanas tiene datos entonces asinamos los datos de las estaciones con latitudes y altitudes semejantes a la estación de interés. La justificación de esto es que la temperatura está fuertemente correlacionada con ambas variables. Para eso escribimos la siguiente función `encontrar_latsalts`.

In [27]:
def encontrar_latsalts(y,tol_lat,tol_alt):
    indices=[]
    for i in range(251):
        if y[0]-tol_lat<=X[i,0]<=y[0]+tol_lat and y[2]-tol_alt<=X[i,2]<=y[2]+tol_alt:
            indices.append(i)
    return indices

### Ejemplo

In [28]:
k_estaciones_cercanas(X[198,:],6)

[198, 146, 128, 248, 12, 199]

In [29]:
stations.loc[[198, 146, 128,248, 12, 199]]

,station ID,latitude,longitude,elevation,State
198,MXN00014310,20.1167,104.3333,1369.8,AYUTLA
146,MXN00014158,19.9500,104.2667,1339.9,UNION DE TULA
128,MXN00014139,20.0167,104.2833,1329.8,TACOTAN
248,MXN00014395,19.9833,104.1833,1460.0,IXTLAHUACAN DE SANTIAGO
12,MXN00014014,20.2833,104.2500,1478.9,ATENGO (SMN)
199,MXN00014312,20.2000,104.4000,1720.0,CUAUTLA


Ninguna de las 6 estaciones más cercanas a Ayutla tiene datos del 2010, motivo por el cual procedemos a encontrar estaciones cuyos datos de latitud y elevación se encuentren en intervalos que tengan como centros los datos de entrada.

In [30]:
def encontrar_latsalts(y,tol_lat,tol_alt):
    indices=[]
    for i in range(251):
        if y[0]-tol_lat<=X[i,0]<=y[0]+tol_lat and y[2]-tol_alt<=X[i,2]<=y[2]+tol_alt:
            indices.append(i)
    return indices

In [31]:
encontrar_latsalts(X[198,:],0.05,50)

[135, 198, 209]

In [32]:
datos_2010(135)

(365,
 PRCP     2.224110
 TMAX    29.337534
 TMIN    12.440548
 dtype: float64,
 '1962-06-07',
 '2012-01-31',
 STATION         MXN00014146
 NAME       TEOCUITATLAN, MX
 Name: 1962-06-07, dtype: object)

In [33]:
stations.loc[[135, 198]]

,station ID,latitude,longitude,elevation,State
135,MXN00014146,20.0833,103.3667,1369.8,TEOCUITATLAN
198,MXN00014310,20.1167,104.3333,1369.8,AYUTLA


el dato 135 sí tiene información del 2010, así que estos son los valores que le asignamos a Ayutla. 

## 3. Municipios que solo tienen registros de algunos días del año 2010.
Buscamos las fechas del 2010 que carecen de datos. Después, buscamos los datos de climatológicos de estas fechas pero de un año proximo (preferentemente de los años 2009 o 2011). Con estos rellenamos los datos de los días del 2010 que no tienen registros.

### Ejemplo 

La estación de Corrinchis, que pertence al municipio de Mascota, es un ejemplo de este tipo de estaciones:

In [34]:
lista_de_datos[32]

,STATION,LATITUDE,LONGITUDE,ELEVATION,NAME,PRCP,PRCP_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES
DATE,,,,,,,,,,,
1961-01-01,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",250.0,",,m",61.0,",,m"
1961-01-02,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",250.0,",,m",39.0,",,m"
1961-01-03,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",250.0,",,m",28.0,",,m"
1961-01-04,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",272.0,",,m",33.0,",,m"
1961-01-05,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",267.0,",,m",39.0,",,m"
...,...,...,...,...,...,...,...,...,...,...,...
2012-01-27,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",NaN,NaN,283.0,",,m",0.0,",,m"
2012-01-28,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",256.0,",,m",17.0,",,m"
2012-01-29,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",250.0,",,m",28.0,",,m"


Como vemos abajo solo tiene 335 registros del año 2010:

In [35]:
datos_2010(32)

(335,
 PRCP     4.222754
 TMAX    28.290390
 TMIN     9.623054
 dtype: float64,
 '1961-01-01',
 '2012-01-31',
 STATION          MXN00014035
 NAME       CORRINCHIS II, MX
 Name: 1961-01-01, dtype: object)

A continuación buscamos cuáles son exactamente los días del 2010 que no tienen datos:

In [36]:
from datetimerange import DateTimeRange
rango=pd.date_range('1/1/2010', end='31/12/2010')
rango.index=rango

In [37]:
missing_dates = rango.index[~rango.index.isin(lista_de_datos[32].loc['2010-01-01':'2010-12-31'].index)]

In [38]:
missing_dates

DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08', '2010-01-09', '2010-01-10', '2010-01-11',
               '2010-01-12', '2010-01-13', '2010-01-14', '2010-01-15',
               '2010-01-16', '2010-01-17', '2010-01-18', '2010-01-19',
               '2010-01-20', '2010-01-21', '2010-01-22', '2010-01-23',
               '2010-01-24', '2010-01-25', '2010-01-26', '2010-01-27',
               '2010-01-28', '2010-01-29', '2010-04-27', '2010-04-28',
               '2010-04-29', '2010-04-30'],
              dtype='datetime64[ns]', freq=None)

In [39]:
len(missing_dates)

30

A veces los años inmediatos, 2009 o 2011, tampoco tienen los registros de estas fechas. En esos casos tomamos el año más proximo con estos datos. Por ejemplo, para Corrinchis, se utilizaron algunos datos del 2004 y otros del 2009.

In [40]:
lista_de_datos[32].loc['2004-01-04':'2004-01-29']

,STATION,LATITUDE,LONGITUDE,ELEVATION,NAME,PRCP,PRCP_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES
DATE,,,,,,,,,,,
2004-01-04,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",261.0,",,m",50.0,",,m"
2004-01-05,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",267.0,",,m",44.0,",,m"
2004-01-06,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",267.0,",,m",56.0,",,m"
2004-01-07,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",267.0,",,m",89.0,",,m"
2004-01-08,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",267.0,",,m",106.0,",,m"
2004-01-09,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",267.0,",,m",100.0,",,m"
2004-01-10,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",157.0,",,m",256.0,",,m",100.0,",,m"
2004-01-11,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",119.0,",,m",261.0,",,m",83.0,",,m"
2004-01-12,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",200.0,",,m",89.0,",,m"


In [41]:
len(lista_de_datos[32].loc['2004-01-04':'2004-01-29'])

26

In [42]:
lista_de_datos[32].loc['2009-04-27':'2009-04-30']

,STATION,LATITUDE,LONGITUDE,ELEVATION,NAME,PRCP,PRCP_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES
DATE,,,,,,,,,,,
2009-04-27,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",339.0,",,m",78.0,",,m"
2009-04-28,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",350.0,",,m",72.0,",,m"
2009-04-29,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",350.0,",,m",83.0,",,m"
2009-04-30,MXN00014035,20.5,-104.7667,1266.7,"CORRINCHIS II, MX",0.0,",,m",350.0,",,m",78.0,",,m"


In [43]:
len(lista_de_datos[32].loc['2009-04-27':'2009-04-30'])

4

El relleno de la estación 32 es el promedio de los datos faltantes.

In [44]:
relleno_32=(lista_de_datos[32].loc['2004-01-04':'2004-01-29',['PRCP','TMAX','TMIN']].mean()*26+lista_de_datos[32].loc['2009-04-27':'2009-04-30',['PRCP','TMAX','TMIN']].mean()*4)/(30*10)

In [45]:
relleno_32

PRCP     3.243333
TMAX    24.273333
TMIN     6.300000
dtype: float64

In [46]:
numdatos2010, datosmeteorologicos2010, primer_dia, ultimo_dia, estacion_nombre=datos_2010(32)

Los valores finales de precipitación, temperatura máxima y mínima que vamos a registrar para el municipio de Mascota es el promedio de los datos correspondientes al año 2010 y el relleno. 

In [47]:
(relleno_32*30+datosmeteorologicos2010*335)/365

PRCP     4.142254
TMAX    27.960221
TMIN     9.349926
dtype: float64

Esta fue la parte más laboriosa del proyecto, puesto que se tenía que buscar estación por estación cuántos datos había del año 2010 y rellenarlos con datos de otros años si hacía falta, además de investigar a qué municipio pertenecían las estaciones meteorólogicas, pues, en muchos casos, el nombre de la estación no coincide con el nombre del municipio. Los promedios anuales que obtuvimos en esta fase, según los criterios anteriormente mencionados, se encuentran en la hoja de cálculo `DatosMeteorologicosJalisco2010.xlsx` 